In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skdemo
from skimage.feature import (match_descriptors, corner_harris,
                             corner_peaks, ORB, plot_matches)


In [19]:
img1=cv2.imread("bag\original dataset\shirt19.jpg",0)
img2 = cv2.imread("bag\original dataset\collared_tees/c3.jpg",0)

# n_keypoints e orb er point gulo assign kor

descriptor_extractor = ORB(n_keypoints=500) 

descriptor_extractor.detect_and_extract(img1)
keypoints1 = descriptor_extractor.keypoints
descriptors1 = descriptor_extractor.descriptors

descriptor_extractor.detect_and_extract(img2)
keypoints2 = descriptor_extractor.keypoints
descriptors2 = descriptor_extractor.descriptors


# Match descriptors between left/right images and the center
matches01 = match_descriptors(descriptors1, descriptors2, cross_check=True)


fig, ax = plt.subplots(1, 1, figsize=(15, 12))

plt.gray()

plot_matches(ax, img1, img2, keypoints1, keypoints2, matches01)
ax.axis('off')


plt.show()


In [20]:
from skimage.transform import ProjectiveTransform
from skimage.measure import ransac

# Select keypoints from 
#   * source (image to be registered): pano0
#   * target (reference image): pano1, our middle frame registration target
src = keypoints1[matches01[:, 0]][:, ::-1]
dst = keypoints2[matches01[:, 1]][:, ::-1]

model_robust01, inliers01 = ransac((src, dst), ProjectiveTransform,
                                   min_samples=4, residual_threshold=1, max_trials=300)


fig, ax = plt.subplots(1, 1, figsize=(15, 12))

# Best match subset for pano0 -> pano1
plot_matches(ax, img1, img2, keypoints1, keypoints2, matches01[inliers01])

ax.axis('off');
plt.show()